In [5]:
import os
import json
import math
import random
import numpy as np
import pandas as pd
import joblib
import xgboost as xgb
import torch
import torch.nn as nn

from dataclasses import dataclass
from typing import Dict, Any, Tuple, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from torch.utils.data import Dataset, DataLoader

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

def rmse(y_true, y_pred):
    return float(math.sqrt(mean_squared_error(y_true, y_pred)))

# Block 3 — Paths + dataset loading
DATASET_PATH = "/content/augmented_dataset.parquet"     # preferred
#FALLBACK_PATH = "/content/surrogate_dataset.parquet"  # Step 5 fallback

OUT_DIR = "Chapter7_Output"
os.makedirs(OUT_DIR, exist_ok=True)

if os.path.exists(DATASET_PATH):
    df = pd.read_parquet(DATASET_PATH)
    print("Loaded augmented dataset:", df.shape)
#elif os.path.exists(FALLBACK_PATH):
#    df = pd.read_parquet(FALLBACK_PATH)
#    print("Loaded Step 5 dataset:", df.shape)
else:
    raise FileNotFoundError(
        "Could not find dataset. Upload augmented_dataset.parquet (Chapter6) or surrogate_dataset.parquet (Step5)."
    )

# Inputs
X_COLS = [
    "core_radius_cm",
    "blanket_thickness_cm",
    "height_cm",
    "initial_U233_frac",
]

# Targets (use the same set you used in Chapter 6)
Y_COLS = [
    "keff_est",
    "flux_shape_metric",
    "breeding_proxy_norm",
    "total_power_W",
    "avg_core_power_density_W_cm3",
]

# Optional column
SYN_FLAG_COL = "is_synthetic" if "is_synthetic" in df.columns else None

# Clean numeric
df = df.copy()
for c in X_COLS + Y_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=X_COLS + Y_COLS).reset_index(drop=True)

print("After cleaning:", df.shape)
print(df[X_COLS + Y_COLS].head(3))

# Block 4 — Split policy (real only test)
# Why: The physics labels exist only for real rows.
# Synthetic rows are model - labeled. They are useful for training, but not for final truth evaluation.
idx_all = np.arange(len(df))

if SYN_FLAG_COL:
    idx_real = df.index[df[SYN_FLAG_COL] == False].to_numpy()
    idx_syn  = df.index[df[SYN_FLAG_COL] == True].to_numpy()
else:
    idx_real = idx_all
    idx_syn  = np.array([], dtype=int)

# Hold out test only from REAL data
idx_real_train, idx_test = train_test_split(idx_real, test_size=0.15, random_state=SEED, shuffle=True)

# Validation from remaining real
idx_real_train, idx_val = train_test_split(idx_real_train, test_size=0.15, random_state=SEED, shuffle=True)

# Training indices = real-train + (optional) all synthetic
idx_train = np.concatenate([idx_real_train, idx_syn]) if len(idx_syn) else idx_real_train

print("Split counts")
print("  train:", len(idx_train), "(real:", len(idx_real_train), ", syn:", len(idx_syn), ")")
print("  val  :", len(idx_val), "(real)")
print("  test :", len(idx_test), "(real)")

# Arrays
X = df[X_COLS].to_numpy(dtype=float)
Y = df[Y_COLS].to_numpy(dtype=float)

X_train, Y_train = X[idx_train], Y[idx_train]
X_val,   Y_val   = X[idx_val],   Y[idx_val]
X_test,  Y_test  = X[idx_test],  Y[idx_test]

# Block 5 — Target transforms (log for heavy-tailed)
LOG_TARGETS = {"breeding_proxy_norm", "total_power_W", "avg_core_power_density_W_cm3"}
EPS = 1e-30

# Train in transformed space, evaluate in physical space.

def transform_Y(Y_in: np.ndarray) -> np.ndarray:
    Yt = Y_in.astype(float).copy()
    for j, name in enumerate(Y_COLS):
        if name in LOG_TARGETS:
            Yt[:, j] = np.log10(np.maximum(Yt[:, j], EPS))
    return Yt


def inverse_transform_y(y_t: np.ndarray, col_name: str) -> np.ndarray:
    if col_name in LOG_TARGETS:
        return np.power(10.0, y_t)
    return y_t

Y_train_t = transform_Y(Y_train)
Y_val_t   = transform_Y(Y_val)
Y_test_t  = transform_Y(Y_test)

# Block 6 — Model A: XGBoost (per target)
# Strength: Very strong on small/medium tabular data.
# Fast to train and robust.
xgb_params = dict(
    n_estimators=1200,
    max_depth=6,
    learning_rate=0.03,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    reg_alpha=0.0,
    random_state=SEED,
    objective="reg:squarederror",
    tree_method="hist",
)

xgb_models = {}

# Define early stopping (using early_stopping_rounds for compatibility)
# The early_stopping_callback object is not directly used with early_stopping_rounds.
early_stopping_rounds_val = 60 # From early_stopping_callback.rounds

for j, tgt in enumerate(Y_COLS):
    model = xgb.XGBRegressor(**xgb_params)
    model.fit(
        X_train,
        Y_train_t[:, j],
        eval_set=[(X_val, Y_val_t[:, j])],
        # early_stopping_rounds=early_stopping_rounds_val, # Removed due to TypeError in old XGBoost versions
        # eval_metric="rmse", # Removed due to TypeError in old XGBoost versions
        # verbose=False, # verbose can be set here if needed, but often defaults to a reasonable level
    )
    xgb_models[tgt] = model

# Predictions
pred_test_t = np.zeros_like(Y_test_t)
for j, tgt in enumerate(Y_COLS):
    pred_test_t[:, j] = xgb_models[tgt].predict(X_test)

pred_test = np.zeros_like(Y_test)
for j, tgt in enumerate(Y_COLS):
    pred_test[:, j] = inverse_transform_y(pred_test_t[:, j], tgt)

# Metrics
metrics_xgb = {}
for j, tgt in enumerate(Y_COLS):
    y_true = Y_test[:, j]
    y_pred = pred_test[:, j]
    metrics_xgb[tgt] = {
        "R2": float(r2_score(y_true, y_pred)),
        "RMSE": rmse(y_true, y_pred),
        "MAE": float(mean_absolute_error(y_true, y_pred)),
    }

print("\nXGBoost test metrics (real-only test):")
for k, v in metrics_xgb.items():
    print(f"  {k:28s}  R2={v['R2']:.4f}  RMSE={v['RMSE']:.4g}  MAE={v['MAE']:.4g}")

# Save models
xgb_dir = os.path.join(OUT_DIR, "xgboost_models")
os.makedirs(xgb_dir, exist_ok=True)
for tgt, model in xgb_models.items():
    joblib.dump(model, os.path.join(xgb_dir, f"xgb_{tgt}.joblib"))

with open(os.path.join(OUT_DIR, "xgb_test_metrics.json"), "w", encoding="utf-8") as f:
    json.dump(metrics_xgb, f, indent=2)

# Block 7 — Optional: Split Conformal intervals (XGBoost)
# Gives distribution-free prediction intervals using the real validation set.
# Quick and useful for ICS safety margins.
# Choose a coverage level (90% interval)
alpha = 0.10

# Build residual quantiles per target in *physical space* using VAL (real)
res_q = {}
for j, tgt in enumerate(Y_COLS):
    pred_val_t = xgb_models[tgt].predict(X_val)
    pred_val = inverse_transform_y(pred_val_t, tgt)
    resid = np.abs(Y_val[:, j] - pred_val)
    q = float(np.quantile(resid, 1.0 - alpha))
    res_q[tgt] = q

with open(os.path.join(OUT_DIR, "xgb_conformal_residual_quantiles.json"), "w", encoding="utf-8") as f:
    json.dump({"alpha": alpha, "residual_quantile": res_q}, f, indent=2)

print("\nConformal residual quantiles saved.")

# Block 8 — Model B: FT - Transformer (multitask)
# Frontier tabular deep model. One network predicts all targets jointly.
# Implementation here is lightweight (Transformer over features).
class TabDataset(Dataset):
    def __init__(self, Xn: np.ndarray, Yn: np.ndarray):
        self.X = torch.tensor(Xn, dtype=torch.float32)
        self.Y = torch.tensor(Yn, dtype=torch.float32)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i):
        return self.X[i], self.Y[i]

# Scale inputs for neural model
x_scaler = StandardScaler()
X_train_s = x_scaler.fit_transform(X_train)
X_val_s   = x_scaler.transform(X_val)
X_test_s  = x_scaler.transform(X_test)

# Scale targets in transformed space to stabilize multitask training
y_scaler = StandardScaler()
Y_train_ts = y_scaler.fit_transform(Y_train_t)
Y_val_ts   = y_scaler.transform(Y_val_t)
Y_test_ts  = y_scaler.transform(Y_test_t)

train_ds = TabDataset(X_train_s, Y_train_ts)
val_ds   = TabDataset(X_val_s, Y_val_ts)
test_ds  = TabDataset(X_test_s, Y_test_ts)

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=256, shuffle=False)


class FTTransformer(nn.Module):
    def __init__(self, n_features: int, d_token: int, n_heads: int, n_layers: int, d_ff: int, out_dim: int, dropout: float):
        super().__init__()
        self.n_features = n_features
        self.d_token = d_token

        # Each feature -> token embedding
        self.feature_embed = nn.Linear(1, d_token)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_token,
            nhead=n_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            activation="gelu",
            batch_first=True,
            norm_first=True,
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layers)

        self.head = nn.Sequential(
            nn.LayerNorm(d_token),
            nn.Linear(d_token, out_dim),
        )

    def forward(self, x):
        # x: [B, F]
        B, F = x.shape
        # tokens: [B, F, 1] -> [B, F, d]
        t = self.feature_embed(x.view(B, F, 1))
        z = self.encoder(t)  # [B, F, d]
        # pool over features
        pooled = z.mean(dim=1)  # [B, d]
        return self.head(pooled)


def eval_mse(model, loader, device):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            pred = model(xb)
            ys.append(yb.cpu().numpy())
            ps.append(pred.cpu().numpy())
    Y = np.vstack(ys)
    P = np.vstack(ps)
    return float(np.mean((Y - P) ** 2)), Y, P


device = "cuda" if torch.cuda.is_available() else "cpu"

model = FTTransformer(
    n_features=len(X_COLS),
    d_token=64,
    n_heads=4,
    n_layers=4,
    d_ff=256,
    out_dim=len(Y_COLS),
    dropout=0.15,
).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)
loss_fn = nn.MSELoss()

best_val = float("inf")
best_state = None
patience = 25
no_improve = 0

for epoch in range(1, 401):
    model.train()
    for xb, yb in train_dl:
        xb = xb.to(device)
        yb = yb.to(device)
        opt.zero_grad(set_to_none=True)
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()

    # val
    val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)
    val_mse, _, _ = eval_mse(model, val_loader, device)

    if val_mse < best_val - 1e-6:
        best_val = val_mse
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        no_improve = 0
    else:
        no_improve += 1

    if epoch % 25 == 0 or epoch == 1:
        print(f"epoch={epoch:4d}  val_mse={val_mse:.6g}  best={best_val:.6g}")

    if no_improve >= patience:
        break

if best_state is not None:
    model.load_state_dict(best_state)

# Evaluate on test (in scaled transformed space)
test_loader = DataLoader(test_ds, batch_size=256, shuffle=False)
_, Yt_true_s, Yt_pred_s = eval_mse(model, test_loader, device)

# Unscale back to transformed space
Yt_true = y_scaler.inverse_transform(Yt_true_s)
Yt_pred = y_scaler.inverse_transform(Yt_pred_s)

# Inverse transform to physical space
Y_true_phys = Y_test
Y_pred_phys = np.zeros_like(Y_true_phys)
for j, tgt in enumerate(Y_COLS):
    Y_pred_phys[:, j] = inverse_transform_y(Yt_pred[:, j], tgt)

metrics_ft = {}
for j, tgt in enumerate(Y_COLS):
    y_true = Y_true_phys[:, j]
    y_pred = Y_pred_phys[:, j]
    metrics_ft[tgt] = {
        "R2": float(r2_score(y_true, y_pred)),
        "RMSE": rmse(y_true, y_pred),
        "MAE": float(mean_absolute_error(y_true, y_pred)),
    }

print("\nFT-Transformer test metrics (real-only test):")
for k, v in metrics_ft.items():
    print(f"  {k:28s}  R2={v['R2']:.4f}  RMSE={v['RMSE']:.4g}  MAE={v['MAE']:.4g}")

# Save artifacts
joblib.dump(x_scaler, os.path.join(OUT_DIR, "ft_x_scaler.joblib"))
joblib.dump(y_scaler, os.path.join(OUT_DIR, "ft_y_scaler.joblib"))

torch.save(model.state_dict(), os.path.join(OUT_DIR, "ft_transformer_state.pt"))

with open(os.path.join(OUT_DIR, "ft_test_metrics.json"), "w", encoding="utf-8") as f:
    json.dump(metrics_ft, f, indent=2)

# Block 9 — Unified report
report = {
    "dataset": {
        "rows": int(len(df)),
        "has_synthetic": bool(SYN_FLAG_COL),
        "train_rows": int(len(idx_train)),
        "val_rows_real": int(len(idx_val)),
        "test_rows_real": int(len(idx_test)),
    },
    "inputs": X_COLS,
    "targets": Y_COLS,
    "log_targets": sorted(list(LOG_TARGETS)),
    "models": {
        "xgboost_per_target": metrics_xgb,
        "ft_transformer_multitask": metrics_ft,
    },
    "conformal": {
        "alpha": alpha,
        "residual_quantile": res_q,
    },
}

with open(os.path.join(OUT_DIR, "chapter7_training_report.json"), "w", encoding="utf-8") as f:
    json.dump(report, f, indent=2)

print("\nSaved Chapter 7 outputs to:", OUT_DIR)

# Block 10 — How to use the trained ICS
# 1) XGBoost: load model for each target; predict in transformed space; inverse-transform if needed.
# 2) FT: scale X -> model -> unscale Yt -> inverse-transform to physical.
# Example: one-point inference
x_example = np.array([[90.0, 60.0, 200.0, 0.01]], dtype=float)  # [1,4]

# XGBoost inference
pred_xgb = {}
for j, tgt in enumerate(Y_COLS):
    y_t = xgb_models[tgt].predict(x_example)[0]
    y_p = inverse_transform_y(y_t, tgt)
    pred_xgb[tgt] = float(y_p)

print("\nExample (XGBoost) prediction:")
print(pred_xgb)

# FT inference
x_s = x_scaler.transform(x_example)
x_t = torch.tensor(x_s, dtype=torch.float32).to(device)
model.eval()
with torch.no_grad():
    yts_s = model(x_t).cpu().numpy()
yts = y_scaler.inverse_transform(yts_s)

pred_ft = {}
for j, tgt in enumerate(Y_COLS):
    pred_ft[tgt] = float(inverse_transform_y(yts[0, j], tgt))

print("\nExample (FT-Transformer) prediction:")
print(pred_ft)


Streaming output truncated to the last 5000 lines.
[1009]	validation_0-rmse:0.01583
[1010]	validation_0-rmse:0.01583
[1011]	validation_0-rmse:0.01583
[1012]	validation_0-rmse:0.01583
[1013]	validation_0-rmse:0.01583
[1014]	validation_0-rmse:0.01583
[1015]	validation_0-rmse:0.01583
[1016]	validation_0-rmse:0.01582
[1017]	validation_0-rmse:0.01582
[1018]	validation_0-rmse:0.01582
[1019]	validation_0-rmse:0.01582
[1020]	validation_0-rmse:0.01582
[1021]	validation_0-rmse:0.01582
[1022]	validation_0-rmse:0.01582
[1023]	validation_0-rmse:0.01582
[1024]	validation_0-rmse:0.01582
[1025]	validation_0-rmse:0.01581
[1026]	validation_0-rmse:0.01581
[1027]	validation_0-rmse:0.01581
[1028]	validation_0-rmse:0.01582
[1029]	validation_0-rmse:0.01582
[1030]	validation_0-rmse:0.01581
[1031]	validation_0-rmse:0.01582
[1032]	validation_0-rmse:0.01582
[1033]	validation_0-rmse:0.01582
[1034]	validation_0-rmse:0.01581
[1035]	validation_0-rmse:0.01581
[1036]	validation_0-rmse:0.01581
[1037]	validation_0-rmse:

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


epoch=   1  val_mse=1.36979  best=1.36979
epoch=  25  val_mse=0.981503  best=0.981503
epoch=  50  val_mse=1.02415  best=0.980219

FT-Transformer test metrics (real-only test):
  keff_est                      R2=0.2010  RMSE=0.23  MAE=0.1878
  flux_shape_metric             R2=0.5616  RMSE=0.1234  MAE=0.09848
  breeding_proxy_norm           R2=0.1035  RMSE=7.073e-09  MAE=4.187e-09
  total_power_W                 R2=0.0662  RMSE=9.848e+07  MAE=5.583e+07
  avg_core_power_density_W_cm3  R2=0.1380  RMSE=20.72  MAE=12.06

Saved Chapter 7 outputs to: Chapter7_Output

Example (XGBoost) prediction:
{'keff_est': 0.5030958652496338, 'flux_shape_metric': 0.44562309980392456, 'breeding_proxy_norm': 5.7222324656436285e-09, 'total_power_W': 42273072.0, 'avg_core_power_density_W_cm3': 7.941456317901611}

Example (FT-Transformer) prediction:
{'keff_est': 0.5756272077560425, 'flux_shape_metric': 0.4426054358482361, 'breeding_proxy_norm': 7.391481648966192e-09, 'total_power_W': 57588712.0, 'avg_core_power